In [9]:
import pandas as pd
from numpy import *
import numpy as np
import scipy.optimize as opt
from scipy.spatial.distance import minkowski, chebyshev
import time

archivo = str(input("Escriba el nombre del archivo con el que se va a trabajar: "))
algoritmo = input("Escriba con algoritmo desea trabajar ('TOPSIS Clasico' - 'TOPSIS con intervalos' - 'UWTOPSIS'): ")
normalizacion = input("Como desea normalizar los datos ('euclidiana' - 'minmax' - 'gauss' - 'none'): ")

data1= pd.read_excel(archivo)
data = data1.iloc[:, 1:]


def normalization(data, norm): 
    
    if norm == 'euclidiana':
        data_norm = data.apply(lambda x: x/linalg.norm(x))
    elif norm == 'minmax':
        data_norm = data.apply(lambda x: (x-min(x))/(max(x)-min(x)))
    elif norm == 'gauss':
        data_norm = data.apply(lambda x: 1/(std(x)*sqrt(2*pi))*exp(-1/2*((x-mean(x))/std(x))**2))
    elif norm == 'none':
        data_norm = data
    
    return data_norm

def normalization_inter(data, norm):
    
    if norm == 'euclidiana':
        for i in range(0,len(data.columns), 2):
            norma = sqrt(sum(data.iloc[:,i]**2) + sum(data.iloc[:,i+1]**2))
            data.iloc[:,i] = data.iloc[:,i] / norma
            data.iloc[:,i+1] = data.iloc[:,i+1] / norma

    return data

def multiplicar_pesos(df, pesos):
    
    for i in range(0,df.shape[1]): 
        df.iloc[:,i] = df.iloc[:,i]*pesos[i]

    return df

def multiplicar_pesos_inter(data, pesos):
    
    con=0
    for i in range(0,len(data.columns), 2):
        data.iloc[:,i] = data.iloc[:,i]*(pesos[con]/2)
        data.iloc[:,i+1] = data.iloc[:,i+1]*(pesos[con]/2)
        con=con+1
    
    return data
    
def get_ideals(data, directions, forceideal, J):
    
    if forceideal:
        Ideal = [int(i == 'max') for i in directions]
        Antiideal = [int(i == 'min') for i in directions]
    else:
        pos_max = [int(i == 'max') for i in directions]
        pos_min = [int(i == 'min') for i in directions]
        col_max = data.apply(lambda z: max(z))
        col_min = data.apply(lambda z: min(z))
        Ideal = array(col_max)*array(pos_max) + array(col_min)*array(pos_min)
        Antiideal = array(col_max)*(1-array(pos_max)) + array(col_min)*(1-array(pos_min))
        
    return Ideal, Antiideal

def get_ideals_inter(data, directions):
    
    ideal = []
    antiideal = []
    con = 0
    for i in range(0,len(data.columns), 2):
    
        opciones = [] 
    
        if directions[con] == 'max':
            opciones.append(max(data.iloc[:,i]))
            opciones.append(max(data.iloc[:,i+1]))
            opciones.append(min(data.iloc[:,i]))
            opciones.append(min(data.iloc[:,i+1]))
        
            ideal.append(max(opciones))
            antiideal.append(min(opciones))
        
        else:
        
            opciones.append(max(data.iloc[:,i]))
            opciones.append(max(data.iloc[:,i+1]))
            opciones.append(min(data.iloc[:,i]))
            opciones.append(min(data.iloc[:,i+1]))
        
            ideal.append(min(opciones))
            antiideal.append(max(opciones))
    
    return ideal, antiideal

def distancia(df, opcion):
    
    distancia = []
    for i in range (len(df.index)):
        d = linalg.norm(df.iloc[i,:] - opcion)
        distancia.append(d)
        
    return distancia

def d_mas_inter(data,ideal):
    
    dmas=[]

    for i in range(len(data.index)):
        suma_cuadrados = []
        con = 0
        for x in range(0,len(data.iloc[i,]),2):
            suma_cuadrados.append((data.iloc[i,x]-ideal[con])**2)
            con = con+1
    
        dmas.append(sqrt(sum(suma_cuadrados)))
    
    return dmas

def d_menos_inter(data,antiideal):
    
    dmenos=[]

    for i in range(len(data.index)):
        suma_cuadrados = []
        con = 0
        for x in range(0,len(data.iloc[i,]),2):
            suma_cuadrados.append((data.iloc[i,x+1]-antiideal[con])**2)
            con = con+1
    
        dmenos.append(sqrt(sum(suma_cuadrados)))
    
    return dmenos
    
def puntuacion(dmas,dmenos):
    
    r_i=[]
    for i in range(len(dmas)):
        r_i.append(dmenos[i]/(dmas[i]+dmenos[i]))
    
    return r_i

def distance(x, v, p):
    
    d = minkowski(x, v, p)
    
    return d

def R(data, Ideal, Antiideal, p):
    
    d_Ideal = array([distance(x, Ideal, p) for x in data])
    d_Antiideal = array([distance(x, Antiideal, p) for x in data])
    R = d_Antiideal/(d_Ideal + d_Antiideal)
    
    return R

def R_i(w, data, Ideal, Antiideal, p, i, optimal_mode, J):
    
    data_norm = w*data
    Ideal = array(w)*Ideal
    Antiideal = array(w)*Antiideal
    r = R(data_norm, Ideal, Antiideal, p)
    
    if optimal_mode == 'min':
        r_i = r[i]
    else:
        r_i = -r[i]
        
    return r_i

def data_df(data, data1):
    
    result = pd.concat([data1, data], axis=1)
    
    return result

def data_df_uw(data,data1):
    
    df1 = pd.DataFrame(data)
    result = pd.concat([data1, df1], axis=1)
    
    return result

def ordenar_lista(r_i):
    
    lista_ordenada = sorted(enumerate(r_i), key=lambda x: x[1], reverse=True)
    nuevos_indices = [i[0] for i in lista_ordenada]

    return nuevos_indices

def optimize_TOPSIS(data, Ideal, Antiideal, p, L, U, optimal_mode, display, I, J):
    
    bounds = [(l,u) for l, u in zip(L, U)]
    constraints = ({'type': 'ineq', 'fun': lambda w: 1-sum(w)},
                   {'type': 'ineq', 'fun': lambda w: sum(w)-1},)
    
    # Optimizing the R-score according to the optimal_mode
    r = []
    w = []
    for i in range(I):
        id_max = argmax(abs(data[i]-Antiideal))
        id_min = argmin(abs(data[i]-Antiideal))
        w0 = (L+U)/2
        if optimal_mode == 'max':
            w0[id_max] = U[id_max]
            w0[id_min] = L[id_min]
        if optimal_mode == 'min':
            w0[id_max] = L[id_max]
            w0[id_min] = U[id_min]
            
        # For Bounded-Constrained problems, we may apply either L-BFGS_B, Powell or TNC methods
        opt_i = opt.minimize(fun = R_i,
                            x0 = w0,
                            jac = None,
                            args = (data, Ideal, Antiideal, p, i, optimal_mode, J),
                            method = 'SLSQP',
                            bounds = bounds,
                            constraints =  constraints,
                            tol = 10**(-8),
                            options = {'disp': display})
        if optimal_mode == 'max':
            opt_i.fun = -opt_i.fun
        r.append(opt_i.fun)
        w.append(opt_i.x)
        
    return r, w

def uwTOPSIS(data, directions, L, U, norm = "euclidean", p = 2, w0=[], alpha = 1/2, forceideal = False, display = False,archivo=False):
    

    I = len(data.index)
    J = len(data.columns)
    data_norm = normalization(data, norm)

    Ideal, Antiideal = get_ideals(data_norm, directions, forceideal, J)
    
    r_min, w_min = optimize_TOPSIS(array(data_norm), Ideal, Antiideal, p, L, U, 'min', display, I, J)
    r_max, w_max = optimize_TOPSIS(array(data_norm), Ideal, Antiideal, p, L, U, 'max', display, I, J)
    uwTOPSIS = [(1-alpha)*m + alpha*M for m, M in zip(r_min,r_max)]
    
    scores = {'R_min': r_min, 'R_max': r_max, 'uwTOPSIS': uwTOPSIS}
    output_uwTOPSIS = {'Ranking': scores, 'Weights_min': w_min, 'Weights_max': w_max}
        
    return output_uwTOPSIS

if algoritmo == 'TOPSIS Clasico':
    num_criterios = int(input("Introduzca con cuantos criterios se va a trabajar: "))
    
    pesos = []
    for i in range (num_criterios):
        elemento = float(input("Ingrese el peso que desea asignarle al criterio número {}: ".format(i+1)))
        pesos.append(elemento)
    
    directions = []
    for i in range (num_criterios):
        elemento = input("Indique si quieres maximizar ('max') o minimizar ('min') el criterio número {}: ".format(i+1))
        directions.append(elemento)
        
    if sum(pesos) != 1:
        raise ValueError('[!] La suma de sus pesos no es igual a 1 .')
    

    
    else:
        
        inicio = time.time()
        data_norm = normalization(data,normalizacion)
        datos_pesos = multiplicar_pesos(data_norm,pesos)
        ideales, antiideales = get_ideals(datos_pesos,directions,False,len(data.columns))
        dmas = distancia(datos_pesos,ideales)
        dmenos = distancia(datos_pesos,antiideales)
        r_i = puntuacion(dmas,dmenos)
        
        fin = time.time()
    
        tiempo_algoritmo = fin -inicio
        
        excel = input("Deseas crear un archivo excel con todos los resultados ('si' - 'no'): ")
        
        df1 = pd.DataFrame()
        df1['Ranking']=r_i
        resultado = data_df(df1,data1)
        r_i_ordenado = ordenar_lista(r_i)
        res_ordenado = resultado.reindex(r_i_ordenado)

        if excel == 'si':
            
            subcadena = '.xlsx'
            archivo = archivo.replace(subcadena,' ')
            res_ordenado.to_excel(archivo + 'topsis_clasico.xlsx', index = True)
            print('El topsis clásico ha tardado en ejecutarse: ',tiempo_algoritmo)
        
        else:
            
            print(res_ordenado)
            print('El topsis clásico ha tardado en ejecutarse: ',tiempo_algoritmo)
            
elif algoritmo == 'TOPSIS con intervalos':
    
    print('\nRecuerde que un criterio esta formado por dos columnas.\n')
    num_criterios = int(input("Introduzca con cuantos criterios se va a trabajar: "))
    pesos = []
    
    print('\nPara el siguinte paso recuerde que cada criterio esta compuesto por dos columnas. \n')
    
    for i in range (num_criterios):
        elemento = float(input("Ingrese el peso que desea asignarle al criterio número {}: ".format(i+1)))
        pesos.append(elemento)
    
    directions = []
    for i in range (num_criterios):
        elemento = input("Indique si quieres maximizar ('max') o minimizar ('min') el criterio número {}: ".format(i+1))
        directions.append(elemento)
    
    if sum(pesos) != 1:
        raise ValueError('[!] La suma de sus pesos no es igual a 1 .')
    
    else:
        
        inicio = time.time()
        data_norm = normalization_inter(data,"euclidiana")
        datos_pesos = multiplicar_pesos_inter(data_norm,pesos)
        ideal, antiideal = get_ideals_inter(datos_pesos,directions)
        dmas = d_mas_inter(datos_pesos, ideal)
        dmenos = d_menos_inter(datos_pesos, antiideal)
        r_i = puntuacion(dmas,dmenos)
        
        fin = time.time()
    
        tiempo_algoritmo = fin -inicio
        
        excel = input("Deseas crear un archivo excel con todos los resultados ('si' - 'no'): ")
        
        df1 = pd.DataFrame()
        df1['Ranking']=r_i
        resultado = data_df(df1,data1)
        r_i_ordenado = ordenar_lista(r_i)
        res_ordenado = resultado.reindex(r_i_ordenado)

        if excel == 'si':
            subcadena = '.xlsx'
            archivo = archivo.replace(subcadena,' ')
            res_ordenado.to_excel(archivo + algoritmo +'.xlsx', index = True)
            print('El topsis con intervalos ha tardado en ejecutarse: ',tiempo_algoritmo)
        
        else:
            
            print(res_ordenado)
            print('El topsis con intervalos ha tardado en ejecutarse: ',tiempo_algoritmo)
        
elif algoritmo == 'UWTOPSIS':
    
    num_criterios = int(input("Introduzca con cuantos criterios se va a trabajar: "))
    
    print('\nAhora va a definir los intervalos de los pesos.\n')
    print('Empecemos por las cotas inferiores.\n')
    
    L = []
    for i in range (num_criterios):
        elemento = float(input("Introduzca el valor mínimo del intervalo correspondiente al criterio número {}: ".format(i+1)))
        L.append(elemento)
        
    print('Ahora vamos a seguir con las cotas superiores.\n')
    
    U = []
    for i in range (num_criterios):
        elemento = float(input("Introduzca el valor máximo del intervalo correspondiente al criterio número {}: ".format(i+1)))
        U.append(elemento)
        
    directions = []
    for i in range (num_criterios):
        elemento = input("Indique si quieres maximizar ('max') o minimizar ('min') el criterio número {}: ".format(i+1))
        directions.append(elemento)
    
    L = np.array(L)
    U = np.array(U)
    
    print('Se esta ejecutando el UWTOPSIS')
    inicio = time.time()
    resultado = uwTOPSIS(data,
                         directions,
                         L,
                         U,
                         norm = normalizacion,
                         p = 2,
                         w0=[],
                         alpha = 1/2,
                         forceideal = False,
                         display = False)
    fin = time.time()
    
    tiempo_algoritmo = fin -inicio
    
    excel = input("Deseas crear un archivo excel con todos los resultados ('si' - 'no'): ")
        
    data_ranking = data_df_uw(resultado['Ranking'],data1.iloc[:,0])
    data_w_min = data_df_uw(resultado['Weights_min'],data1.iloc[:,0])
    data_w_max = data_df_uw(resultado['Weights_max'],data1.iloc[:,0])
    
    r_i_ordenado = ordenar_lista(resultado['Ranking']['uwTOPSIS'])
    res_ordenado = data_ranking.reindex(r_i_ordenado)
    
    data_ranking = data_ranking.reindex(r_i_ordenado)
    data_w_min = data_w_min.reindex(r_i_ordenado)
    data_w_max = data_w_max.reindex(r_i_ordenado)
    
    if excel == 'si':
        
        subcadena = '.xlsx'
        archivo = archivo.replace(subcadena,' ')
        res_ordenado.to_excel(archivo + algoritmo +'_ranking.xlsx', index = True)
        data_w_min.to_excel(archivo + algoritmo +'_cotas_minimas.xlsx', index = True)
        data_w_max.to_excel(archivo + algoritmo +'_cotas_maximas.xlsx', index = True)
        
        print('El UWTOPSIS ha tardado en ejecutarse: ',tiempo_algoritmo)
        
    else:
        
        print(res_ordenado)
        print(data_w_min)
        print(data_w_max)
        print('El UWTOPSIS ha tardado en ejecutarse: ',tiempo_algoritmo)

    

Escriba el nombre del archivo con el que se va a trabajar: AJSA
Escriba con algoritmo desea trabajar ('TOPSIS Clasico' - 'TOPSIS con intervalos' - 'UWTOPSIS'): ASJ
Como desea normalizar los datos ('euclidiana' - 'minmax' - 'gauss' - 'none'): SJS


FileNotFoundError: [Errno 2] No such file or directory: 'AJSA'